In [1]:
import os, sys, email
import numpy as np
import pandas as pd

In [2]:
from email.parser import Parser

path = "enron/maildir/allen-p/inbox/5."

with open(path) as f:
    mail = Parser().parse(f)

In [24]:
cols = ['Path', 'Message-ID', 'Date', 'From', 'To', 'Cc', 'Bcc', 'X-From', 'X-To', 'X-cc', 'X-bcc', 'Subject', 'Message']
df = pd.DataFrame(index=[], columns=cols)

In [25]:
record = {}
for header in cols:
    if header == 'Path':
        record[header] = path
    elif header == 'Message':
        record[header] = mail.get_payload()
    else:
        record[header]=mail.get(header)

In [26]:
df = df.append(record, ignore_index=True)
df.head()

,Path,Message-ID,Date,From,To,Cc,Bcc,X-From,X-To,X-cc,X-bcc,Subject,Message
0,enron/maildir/allen-p/inbox/5.,<2952100.1075855377529.JavaMail.evans@thyme>,"Mon, 17 Dec 2001 08:40:38 -0800 (PST)",steven.matthews@ubspw.com,k..allen@enron.com,None,None,"""Matthews, Steven"" <steven.matthews@ubspw.com>...","Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...",,,Phillip Allen Muni Bond Proposal.xls,"\nPhillip,\n\nThis is the proposal for the 1mm..."
